# Tutorial 01: Conforming Finite Element Method for a Nonlinear Poisson Equation

In this tutorial we extend tutorial 00 in the following ways:

1. Solve a nonlinear stationary partial differential equation (PDE).
2. Use conforming finite element spaces of arbitrary order.
3. Use different types of (conforming) meshes (simplicial, cubed and mixed).
4. Use multiple types of boundary conditions.

Combined with the fact that the implementation works in any dimension (note: it is not claimed to be efficient in high dimension $d>3$) this comprises already a relatively large space of different methods, so the example illustrates the flexibility of PDELab. Moreover, the finite element method developed in this tutorial will serve as a building block for instationary problems, adaptive mesh refinement and
parallel solution in subsequent tutorials. This tutorial depends on tutorial 00 which discusses piecewise linear elements on simplicial elements. It is assumed that you have worked through tutorial 00 before.

## Problem Formulation

Here we consider the following nonlinear Poisson equation with
Dirichlet and Neumann boundary conditions:


\begin{align}\label{eq:ProblemStrong}
-\Delta u + q(u) &= f &&\text{in $\Omega$}, \\
u &= g &&\text{on $\Gamma_D\subseteq\partial\Omega$},\\
-\nabla u\cdot \nu &= j &&\text{on $\Gamma_N=\partial\Omega\setminus\Gamma_D$}.
\end{align}


$\Omega\subset\mathbb{R}^d$ is a domain, $q:\mathbb{R}\to\mathbb{R}$ is a given, possibly
nonlinear function and $f: \Omega\to\mathbb{R}$ is the source term and
$\nu$ denotes the unit outer normal to the domain.

The weak formulation of this problem is derived by multiplication with an appropriate
test function and integrating by parts. This results in the abstract problem:
\begin{equation}
\text{Find $u\in U$ s.t.:} \quad r^{\text{NLP}}(u,v)=0 \quad \forall v\in V,
\label{Eq:BasicBuildingBlock}
\end{equation}

with the continuous residual form
\begin{equation*}
r^{\text{NLP}}(u,v) = \int_\Omega \nabla u \cdot \nabla v + (q(u)-f)v\,dx + \int_{\Gamma_N} jv\,ds
\label{eq:ResidualForm}
\end{equation*}
and the function spaces 
$U= \{v\in H^1(\Omega) \,:\, \text{''$v=g$'' on $\Gamma_D$}\}$
and $V= \{v\in H^1(\Omega) \,:\, \text{''$v=0$'' on $\Gamma_D$}\}$. 
We assume that $q$ is such that this problem has a unique solution.

## Finite Element Method

## Realization in PDELab

TO DO
- [x] is this producing the right output?
- [x] include driver
- [x] include problem.hh
- [x] include explanation of LOP class, same way as problem.hh
- [x] provide solutions for exercise1
- [x] make an exercise for yasp grid implementation
- [x] make subsampling default depending on degree
- [x] make real exercises out of parts for exercise2
- [x] only used parameters should stay in .ini, otherwise very confusing
- [x] dim=3 as exercise,check that it works
- [ ] structured grid factory implementation of yasp grid works with gridview, so can only be visualized through gv
- [ ] keep both yasp grid possibilities?
- [x] dir for images, make sure all paths are still correct within notebook
- [ ] exercise 2: go through exercise/solution code again, what part of solution should be provided?
- [ ] how many parameters set in ini file, how many in notebook?


FINALIZING
- [ ] make sure all headers are back to original versions
- [ ] make sure nitsche LOP has NO alpha boundary term
- [ ] include nonlinpoissonfem

The structure of the code is very similar to that of tutorial 00. Again, all the Dune sources are included through one convenience header:

In [ ]:
#include <dune/jupyter.hh>
#include "nonlinearpoissonfem.hh"
//#include "nitschenonlinearpoissonfem.hh"
#include "problem.hh"

As for the previous tutorial, a number of runtime paramters is provided in a configuration file `tutorial01.ini`, which can be found in the `notebooks/tutorial01` directory. Again the paramters are parsed into a `ParamterTree`.

In [ ]:
Dune::ParameterTree ptree;
Dune::ParameterTreeParser ptreeparser;
ptreeparser.readINITree("tutorial01.ini",ptree);

Ini File not that important for jupyter notebooks

**First, a DUNE grid object is instantiated**

The dimension is set and the refinement is read from the ini file. <a id='dim'> </a>

In [ ]:
// read ini file
const int dim = 3;
const int refinement = ptree.get<int>("grid.refinement");

The first step is again, to instantiate a grid. We use again a two-dimensional, unstructured mesh for this simulation, as known from the previous tutorial. Again, the mesh resolves the unit square $\Omega = [0,1]^2$. The file is specified in the `unitsquare.msh` file that was generated using Gmsh.

In [ ]:
using Grid = Dune::UGGrid<dim>;
//std::string filename = ptree.get("grid.twod.filename",
//                                 "unitsquare.msh");
std::string filename = ptree.get("grid.threed.filename",
                                 "bla.msh");
Dune::GridFactory<Grid> factory;
Dune::GmshReader<Grid>::read(factory,filename,true,true);
std::unique_ptr<Grid> gridp(factory.createGrid());
Dune::Timer timer;
gridp->globalRefine(refinement);
std::cout << "Time for mesh refinement " << timer.elapsed()
          << " seconds" << std::endl;
using GV = Grid::LeafGridView;
using DF = Grid::ctype;
GV gv = gridp->leafGridView();

*Using yasp grid instead of uggrid. Note, that additionally another finite element map needs to be selected.* <font color ='red'> Does the second implementation work? </font>
<a id='yasp'> </a>

In [ ]:
//solution to task 1.2
/*using Grid = Dune::YaspGrid<dim>;
using DF = Grid::ctype;
Dune::FieldVector<DF,dim> L;

L[0] = ptree.get("grid.structured.LX",(double)1.0);
L[1] = ptree.get("grid.structured.LY",(double)1.0);
std::array<int,dim> N;
N[0] = ptree.get("grid.structured.NX",(int)10);
N[1] = ptree.get("grid.structured.NY",(int)10);

std::unique_ptr<Grid> gridp = std::make_unique<Grid>(L,N);
gridp->globalRefine(refinement);
using GV = Grid::LeafGridView;
GV gv=gridp->leafGridView();
*/

In [ ]:
//solution to task 1.2 using the structured grid factory
/*using Grid = Dune::YaspGrid<dim>;
using DF = Grid::ctype;

Dune::FieldVector<DF,dim> lowerleft(0.0);
Dune::FieldVector<DF,dim> upperright(1.0);

const std::array<unsigned int,dim> N = {2,2};
//auto N  = Dune::filledArray<dim, unsigned int>(2);
//N[0] = ptree.get("grid.structured.NX",(int)10);
//N[1] = ptree.get("grid.structured.NY",(int)10);

auto grid = Dune::StructuredGridFactory<Grid>::createCubeGrid(lowerleft, upperright, N);
using GV = Grid::LeafGridView;
GV gv = grid->leafGridView();*/

In [ ]:
//gridp

Here, a polynomial degree can bee chosen:

In [ ]:
const int degree = 1;

In [ ]:
//ug
using FEM = Dune::PDELab::PkLocalFiniteElementMap<GV,DF,double,degree>;

*Finite element map for yasp grid*

In [ ]:
//yasp
//using FEM = Dune::PDELab::QkLocalFiniteElementMap<GV,DF,double,degree>; 

In [ ]:
FEM fem(gv);

### Driver

**The driver instantiates the necessary PDELab classes for solving a nonlinear stationary problem and finally solves the problem**

In [ ]:
// type for calculations
using RF = double; 

The interface of the parameter class is defined by the implementor of the local operator and is not part of PDELab. As shown in tutorial 00 it is perfectly possible to have a local operator without a parameter class.
The following code segment instantiates the problem class which is called `Problem` here (it is explained in detail below)

In [ ]:
// make PDE parameter class
RF eta = ptree.get("problem.eta",(RF)1.0);
Problem<RF> problem(eta);

In [ ]:
eta

Now there are two places where information from the PDE is used in PDELab. First of all we need to have an object that can be used as an argument to `Dune::PDELab::interpolate` to initialize a vector which represents the initial guess and the Dirichlet boundary conditions.
The class `Problem` defines a method which we need to use to define a class with the interface of `Dune::PDELab::GridFunction`.
This is accomplished by the following code:

In [ ]:
auto g = Dune::PDELab::makeGridFunctionFromCallable(
    gv,
    [&](const auto& e, const auto& x) {
        return problem.g(e,x);
    }                                             
);;

Similarly, we need an object that can be passed to `Dune::PDELab::constraints` to fill a constraints container which is used to build a subspace of a function space. Again, the class `Problem` defines such a method which is extracted with a lambda function:

In [ ]:
auto b = Dune::PDELab::makeBoundaryConditionFromCallable(
    gv,
    [&](const auto& i, const auto& x){
        return problem.b(i,x);
    }
);;

The next step is to define the grid function space. This is
exactly the same code as in tutorial 00.

In [ ]:
// Make grid function space
//== Exercise 2 {
using CON = Dune::PDELab::ConformingDirichletConstraints;
// using CON = Dune::PDELab::NoConstraints;
//== }
using VBE = Dune::PDELab::ISTL::VectorBackend<>;
using GFS = Dune::PDELab::GridFunctionSpace<GV,FEM,CON,VBE>;
GFS gfs(gv,fem);
gfs.name("Vh");

Now comes unchanged code to assemble the constraints, instantiate a coefficient vector, making a discrete grid function that can be used for visualization and interpolating the initial guess and Dirichlet boundary conditions:

In [ ]:
// Assemble constraints
//== Exercise 2 {
using CC = typename GFS::template
ConstraintsContainer<RF>::Type;
CC cc;
Dune::PDELab::constraints(b,gfs,cc); // assemble constraints
std::cout << "constrained dofs=" << cc.size() << " of "
        << gfs.globalSize() << std::endl;
// using CC = Dune::PDELab::EmptyTransformation;
//== }

// A coefficient vector
using Z = Dune::PDELab::Backend::Vector<GFS,RF>;
Z z(gfs); // initial value

// Make a grid function out of it
using ZDGF = Dune::PDELab::DiscreteGridFunction<GFS,Z>;
ZDGF zdgf(gfs,z);

// Fill the coefficient vector
Dune::PDELab::interpolate(g,gfs,z);

The next step is to instantiate a local operator, called `NonlinearPoissonFEM`, containing the implementation of the element-wise computations of the finite element method. As explained above the local operator is parametrized by the class 
`Problem`. In addition, also the finite element map is passed as a template parameter for reasons that will become clear below:

In [ ]:
// Make a local operator
//== Exercise 2 {
using LOP = NonlinearPoissonFEM<Problem<RF>,FEM> ;
LOP lop(problem);
// RF stab = ptree.get("fem.stab",(RF)1);
// using LOP = NitscheNonlinearPoissonFEM<Problem<RF>,FEM> ;
// LOP lop(problem,stab);
//== }

Now the grid function space, local operator, matrix backend and
constraints container are used to set up a grid operator facilitating the global residual assembly, Jacobian assembly and matrix-free Jacobian application. The matrix backend is initialized with a guess of the approximate number
of nonzero matrix entries per row.

In [ ]:
// Make a global operator
using MBE = Dune::PDELab::ISTL::BCRSMatrixBackend<>;
MBE mbe((int)pow(1+2*degree,dim));
typedef Dune::PDELab::GridOperator<
GFS,GFS,  /* ansatz and test space */
LOP,      /* local operator */
MBE,      /* matrix backend */
RF,RF,RF, /* domain, range, jacobian field type*/
CC,CC     /* constraints for ansatz and test space */
> GO;
//== Exercise 2 {
GO go(gfs,cc,gfs,cc,lop,mbe);
// GO go(gfs,gfs,lop,mbe);
//== }

In order to prepare for the solution process an appropriate linear solver needs to be selected:

In [ ]:
using LS = Dune::PDELab::ISTLBackend_SEQ_CG_AMG_SSOR<GO> ;
LS ls(100,2);

Since the problem is nonlinear we use the implementation of
Newton's method in PDELab. It provides the inexact Newton method
in the sense that the iterative solution of the linear subproblems is stopped early and uses line search as globalization strategy:

In [ ]:
Dune::PDELab::Newton<GO,LS,Z> newton(go,z,ls);
newton.setReassembleThreshold(0.0); // always reassemble J
newton.setVerbosityLevel(3);        // be verbose
newton.setReduction(1e-10);         // total reduction
newton.setMinLinearReduction(1e-4); // min. red. in lin. solve
newton.setMaxIterations(25);        // limit number of its
newton.setLineSearchMaxIterations(10); // limit line search

Now, finally do all the work and solve the problem:

In [ ]:
newton.apply();

At the end we can write the VTK file with subsampling: <a id='exact'> </a> <font color ='red'> correct default parameter? </font>

In [ ]:
// Write VTK output file
int subsampling = ptree.get("output.subsampling",degree);
Dune::SubsamplingVTKWriter<GV> vtkwriter(gv,Dune::refinementIntervals(subsampling));
using VTKF = Dune::PDELab::VTKGridFunctionAdapter<ZDGF> ;
vtkwriter.addVertexData(std::shared_ptr<VTKF>(new VTKF(zdgf,"fesol")));

*Visualize exact solution*

In [ ]:
//solution to exercise 2.1.3
Z w(gfs); 
Dune::PDELab::interpolate(g,gfs,w); // Lagrange interpolation of exact solution
ZDGF wdgf(gfs,w);

In [ ]:
//solution to exercise 2.1.3
vtkwriter.addVertexData(std::shared_ptr<VTKF>(new VTKF(wdgf,"exact")));

The visualization data can be generated in Jupyter by printing the VTKWriter instance:

In [ ]:
vtkwriter

### Problem.hh

<font color= "red"> Note that the cells are note executable, as the class would not a allow for a seperation of cells. Changes can be made in the file `problem.hh` which is provided in the same folder as this notebook. </font>

The class `Problem` contained in the file `problem.hh` provides all parameter functions for the PDE problem. It is parameterized with the floating point type to be used:

```c++
template<typename Number>
class Problem
```

Its constructor takes a parameter $\eta$ as argument:

```c++
Problem (const Number& eta_) : eta(eta_) {}
```

Now come the parameter functions defining the PDE problem.
First is the nonlinearity $q(u)$:

```c++
Number q (Number u) const
{
  return eta*u*u;
}
```

We also provide the derivative of the function $q$ as a seperate method:

```c++
Number qprime (Number u) const
{
  return 2*eta*u;
}
```

This allows the implementation of an exact Jacobian later (illustrated in tutorial 02) and is actually not needed here as we will use a numerical Jacobian.

Next is the right hand side function $f$ which gets an element `e` and a local coordinate `x` within
the corresponding reference element as a parameter:

```c++
template<typename E, typename X>
Number f (const E& e, const X& x) const
{
  return -2.0*x.size();
}
```

The argument `x` can be expected to be an instance of`Dune::FieldVector` which has a method `size`
giving the number of components of the vector, i.e. the space dimension.

The next method simply called `b` is the boundary condition type function. It should return true if the position given by intersection `i` and a local coordinate `x` within the reference element of the intersection is on the Dirichlet boundary.
In the particular instance here we set $\Gamma_D=\partial\Omega$:

```c++
template<typename I, typename X>
bool b (const I& i, const X& x) const
{
  return true;
}
```

The value of the Dirichlet boundary condition is now defined by the method `g`. <font color="red">As explained above in Section</font> it
is more appropriate to provide a function $u_g$ that can be evaluated
on $\overline{\Omega}$ and gives the value of $g$ on the Dirichlet boundary
and the initial guess for the nonlinear solver on all other points:

```c++
template<typename E, typename X>
Number g (const E& e, const X& x) const
{
  auto global = e.geometry().global(x);
  Number s=0.0;
  for (std::size_t i=0; i<global.size(); i++) s+=global[i]*global[i];
  return s;
}
```

As with the function `f` above the arguments are an element and a local coordinate
in its reference element. Here we evaluate it as $u_g(e,x) = \|\mu_e(x)\|^2$.

Finally, there is a method defining the value of the Neumann boundary condition.
Although there is no Neumann boundary here, the method has to be provided but is
never called. The arguments of the method are the same as for the boundary
condition type function `b`:

```c++
template<typename I, typename X>
Number j (const I& i, const X& x) const
{
  return 0.0;
}
```

### Local Operator

We now turn to how the residual can be evaluated in practice. The residual form \eqref{eq:ResidualForm} can be readily decomposed into elementwise contributions:
\begin{equation*}
r^{\text{NLP}}\left(u,v\right) =  
\sum_{T\in\mathcal{T}_h} \alpha_T^V(u,v) 
  + \sum_{T\in\mathcal{T}_h} \lambda_T^V(v)
 + \sum_{F\in\mathcal{F}_h^{\partial\Omega}}\lambda_F^B(v)
\end{equation*}
with
\begin{align*}
\alpha_T^V(u,v) &= \int_T \nabla u \cdot \nabla v + q(u) v \,dx, &
\lambda_T^V(v) &= - \int_T f v \,dx, &
\lambda_F^B(v) &= \int_{F\cap\Gamma_N} j v\,ds.
\end{align*}

Here $\mathcal{F}_h^{\partial\Omega}$ is the set of intersections of elements with the domain boundary $\partial\Omega$.
The element-wise computations can be classified on the one hand as volume integrals (superscript $V$), boundary integrals (superscript $B$) and skeleton integrals (superscript $S$, to be shown later) and on the
other hand as integrals depending on trial and test functions ($\alpha$-terms) and integrals depending only on test functions ($\lambda$-terms). Here we need three of these six possible combinations.

The three terms can now be evaluated using the techniques introduced in tutorial 00 with the small extension that for general maps $\mu_T$ we have 
$$\nabla w(\mu_T(\hat x)) = J_{\mu_T}^{-1}(\hat x) \hat\nabla \hat w (\hat x)$$
with $J_{\mu_T}(\hat x)$ the Jacobian of $\mu_T$ at point $\hat x$.

The class `NonlinearPoissonFEM` implements the element-wise computations of the finite element method
<font color='red'>introduced in Section `Sec:ElementComputations` above </font>. Evaluation of the residual $R(z)$ is accomplished
by the three types of contributions <font color='red'> shown in equation \eqref{eq:FinalResidualEvaluation}. </font>
In order to make things as simple as possible we chose to implement the evaluation of the Jacobian and the matrix-free Jacobian application with finite differences. 

The definition of class `NonlinearPoissonFEM` starts as follows:

```c++
template<typename Param, typename FEM>
class NonlinearPoissonFEM :
  public Dune::PDELab::
    NumericalJacobianVolume<NonlinearPoissonFEM<Param,FEM> >,
  public Dune::PDELab::
    NumericalJacobianApplyVolume<NonlinearPoissonFEM<Param,FEM> >,
  public Dune::PDELab::FullVolumePattern,
  public Dune::PDELab::LocalOperatorDefaultFlags
``` 

The class is parametrized by a parameter class and a finite element map.
Implementation of element-wise contributions to the Jacobian and matrix-free
Jacobian evaluation is achieved through inheriting from the
classes `NumericalJacobianVolume` and `NumericalJacobianApplyVolume`.
Using the *curiously recurring template pattern* these classes provide 
the corresponding methods without any additional coding effort
based on the `alpha_volume` method explained below.
The other two base classes are the same as in tutorial 00.

The private data members are a cache for evaluation of the basis functions on the reference element:

```c++
typedef typename FEM::Traits::FiniteElementType::
     Traits::LocalBasisType LocalBasis;
  Dune::PDELab::LocalBasisCache<LocalBasis> cache;
```

a reference to the parameter object:

```c++
Param& param;
```

and an integer value controlling the order of the formulas used for numerical quadrature:

```c++
int incrementorder;
```

The public part of the class starts with the definition of the flags controlling
the generic assembly process. The `doPatternVolume` flag specifies that the sparsity pattern of the Jacobian is determined by couplings between degrees of freedom associated with single elements. The corresponding
default pattern assembly method is inherited from the class `FullVolumePattern`:

```c++
enum { doPatternVolume = true };
```

The residual assembly flags indicate that in this local operator we will provide the methods `lambda_volume`, `lambda_boundary` and `alpha_volume`:

```c++
enum { doLambdaVolume = true };
enum { doLambdaBoundary = true };
enum { doAlphaVolume = true };
```

Next comes the constructor taking as an argument a reference to a parameter object and the optional increment of the quadrature order:

```c++
NonlinearPoissonFEM (Param& param_, int incrementorder_=0)
    : param(param_), incrementorder(incrementorder_)
  {}
```

#### Method `lambda_volume`

For any $(T,m)\in C(i)$ we obtain 
\begin{equation*}
\begin{split}
\lambda_T^V(\phi_i) &= - \int_T f \phi_i \,dx = 
- \int_{\hat T} f(\mu_T(\hat x)) p_m^{\hat T}(\hat x) |\text{det} J_{\mu_T}(\hat x)|\, d\hat x .
\end{split}
\end{equation*}
This integral on the reference element is then computed by employing numerical integration of appropriate order.
The evaluation for all test functions with support on element $T$ may be collected in a vector 
\begin{equation*}
(\mathcal{L}_T^V)_m = - \int_{\hat T} f(\mu_T(\hat x)) p_m^{\hat T}(\hat x) 
|\text{det} J_{\mu_T}(\hat x)|\, d\hat x.
\end{equation*}

<font color='red'> This method was also present in the local operator `PoissonP1` in tutorial00. It implements the term $\mathcal{L}_T^V$ and has the interface: </font>

```c++
//! right hand side integral
template<typename EG, typename LFSV, typename R>
void lambda_volume (const EG& eg, const LFSV& lfsv, R& r) const
```

The implementation here uses numerical quadrature of sufficiently high order
which is selected at the beginning of the method:

```c++
auto geo = eg.geometry();
const int order = incrementorder + 2*lfsv.finiteElement().localBasis().order();
auto rule = Dune::PDELab::quadratureRule(geo,order);
```

The DUNE quadrature rules provide a container of quadrature points that can be iterated over:

```c++
for (const auto& ip : rule)
      {
        // evaluate basis functions
        auto& phihat = cache.evaluateFunction(ip.position(), lfsv.finiteElement().localBasis());

        // integrate -f*phi_i
        decltype(ip.weight()) factor = ip.weight()*
          geo.integrationElement(ip.position());
        auto f=param.f(eg.entity(),ip.position());
        for (size_t i=0; i<lfsv.size(); i++)
          r.accumulate(lfsv,i,-f*phihat[i]*factor);
      }
```      

At each quadrature point all basis functions are evaluated. The local function space argument `lfsv` provides all the basis functions on the reference element. Evaluations are cached for each point as the evaluation
may be quite costly, especially for high order. In addition, copying of data is avoided as the cache returns only a reference to the data stored in the cache. The integration factor is the product of the weight of the quadrature point and the value of $|\text{det} J_{\mu_T}(\hat x)|$. The implementation works also
for non-affine element transformation. The quadrature order should be increased by providing a value for `incrementorder` in the constructor.
Then the parameter function can be evaluated and finally the residual contributions for each test function are stored in the result object `r`.

#### Method `lambda_boundary`

For $F\in\mathcal{F}_h^{\partial\Omega}$ with $F\cap\Gamma_N\neq\emptyset$ and $(T_F^-,m)\in C(i)$ we obtain
\begin{equation*}
\begin{split}
\lambda_T^B(\phi_i) &= \int_{F} j v\,ds = 
\int_{\hat F} j(\mu_F(s)) p_m^{\hat T}(\eta_F(s)) 
\sqrt{|\text{det} (J^T_{\mu_F}(s)J_{\mu_F}(s))|} \,ds
\end{split}
\end{equation*}
Because integration is over a face of codimension 1 now, two mappings are involved. The map $\mu_F$ maps the reference element $\hat F$ of $F$ into global coordinates while the map $\eta_F$ maps $\hat F$ into the reference element $\hat T$ of $T$. Also the integration element has to be redefined accordingly.
Again, all contributions of the face $F$ can be collected in a vector:
\begin{equation*}
(\mathcal{L}_T^B)_m = 
\int_{\hat F} j(\mu_F(s)) p_m^{\hat T}(\eta_F(s)) 
\sqrt{|\text{det} J^T_{\mu_T}(s)J_{\mu_T}(s)|} \,ds .
\end{equation*}

<font color = 'red'>The `lambda_boundary` implements the residual contributions due to Neumann boundary conditions. 
It implements the term $\mathcal{L}_T^B$ and has the following interface: </font>

```c++
template<typename IG, typename LFSV, typename R>
void lambda_boundary (const IG& ig, const LFSV& lfsv, R& r) const
 ``` 

The difference to `lambda_volume` is that now an intersection is provided as first argument.

The method begins by evaluating the type of the boundary condition at the midpoint of the edge:

```c++
auto localgeo = ig.geometryInInside();
auto facecenterlocal = referenceElement(localgeo).position(0,0);
bool isdirichlet = param.b(ig.intersection(),facecenterlocal);
```

To that end the center of the reference element of the intersection is computed in the variable`facecenterlocal` before the parameter function can be called.

If the boundary condition type evaluated at the face center is Dirichlet then the complete face is assumed to be part of the Dirichlet boundary:

```c++
// skip rest if we are on Dirichlet boundary
if (isdirichlet) return;
```

It is thus assumed that the mesh resolves all positions where the boundary type changes.

Now that we are on a Neumann boundary an appropriate quadrature rule is selected for integration:

```c++
auto globalgeo = ig.geometry();
const int order = incrementorder + 2*lfsv.finiteElement().localBasis().order();
auto rule = Dune::PDELab::quadratureRule(globalgeo,order);
```

And here is the integral over the face:

```c++
// loop over quadrature points and integrate normal flux
for (const auto& ip : rule)
{
  // quadrature point in local coordinates of element
  auto local = localgeo.global(ip.position());

  // evaluate shape functions (assume Galerkin method)
  auto& phihat = cache.evaluateFunction(local,lfsv.finiteElement().localBasis());

  // integrate j
  decltype(ip.weight()) factor = ip.weight()*globalgeo.integrationElement(ip.position());
  auto j = param.j(ig.intersection(),ip.position());
  for (size_t i=0; i<lfsv.size(); i++)
    r.accumulate(lfsv,i,j*phihat[i]*factor);
}
```      

Every quadrature point on the face needs to be mapped to the reference of the volume element for evaluation of the basis functions. The evaluation uses the basis function cache. Then the integration factor is computed and the contributions for all the test functions are accumulated.

#### Method `alpha_volume`

For any $(T,m)\in C(i)$ we get
\begin{equation*}
\begin{split}
\alpha_T^V(u_h,\phi_i) &= \int_T \nabla u \cdot \nabla \phi_i + q(u) \phi_i \,dx,
= \int_T \sum_j (z)_j \left(\nabla \phi_j \cdot \nabla \phi_i \right) 
+ q\left( \sum_j (z)_j \phi_j \right) \phi_i \,dx,\\
&= \int_{\hat T} \sum_{n} (z)_{g_T(n)} (J_{\mu_T}^{-1}(\hat x) \hat\nabla p_n^{\hat T}(\hat x) )
\cdot (J_{\mu_T}^{-1}(\hat x) \hat\nabla p_m^{\hat T}(\hat x) ) \\
&\hspace{40mm}+ q\left( \sum_n (z)_{g_T(n)} p_n^{\hat T}(\hat x) \right) p_m^{\hat T}(\hat x) 
|\text{det} J_{\mu_T}(\hat x)| \,d\hat x
\end{split}
\end{equation*}
Again contributions for all test functions can be collected in a vector
\begin{equation*}
\begin{split}
(\mathcal{R}_T^V(R_T z))_m &=
\sum_{n} (z)_{g_T(n)} \int_{\hat T} (J_{\mu_T}^{-1}(\hat x) \hat\nabla p_n^{\hat T}(\hat x) )
\cdot (J_{\mu_T}^{-1}(\hat x) \hat\nabla p_m^{\hat T}(\hat x) ) |\text{det} J_{\mu_T}(\hat x)| \,d\hat x\\
&\hspace{30mm}+ \int_{\hat T} q\left( \sum_n (z)_{g_T(n)} p_n^{\hat T}(\hat x) \right) p_m^{\hat T}(\hat x) 
|\text{det} J_{\mu_T}(\hat x)| \,d\hat x
\end{split}
\end{equation*}

<font color = 'red'>This method was already present in tutorial00. It implements the term $\mathcal{R}_T^V(R_T z)$ and its interface is </font>

```c++
template<typename EG, typename LFSU, typename X,
         typename LFSV, typename R>
void alpha_volume (const EG& eg, const LFSU& lfsu, const X& x,
                   const LFSV& lfsv, R& r) const
```

The method starts by extracting the space dimension and the floating point type to be used for computations:

```c++
const int dim = EG::Entity::dimension;
typedef decltype(Dune::PDELab::makeZeroBasisFieldValue(lfsu)) RF;
```

Then a quadrature rule is selected

```c++
auto geo = eg.geometry();
const int order = incrementorder + 2*lfsu.finiteElement().localBasis().order();
auto rule = Dune::PDELab::quadratureRule(geo,order);
```  

and the quadrature loop is started

```c++
for (const auto& ip : rule)
  {
```  

Within the quadrature loop the basis functions are evaluated

```c++
auto& phihat = cache.evaluateFunction(ip.position(),lfsu.finiteElement().localBasis());
```

and the value of $u_h$ at the quadrature point is computed.

```c++
RF u=0.0;
for (size_t i=0; i<lfsu.size(); i++)
  u += x(lfsu,i)*phihat[i];
```

Then the gradients of the basis functions on the reference element are evaluated via the evaluation cache:

```c++
auto& gradphihat = cache.evaluateJacobian(ip.position(), lfsu.finiteElement().localBasis());
```

Now the gradients need to be transformed from the reference element to the transformed element by multiplication with $J_{\mu_T}^{-1}(\hat x)$:

```c++
// transform gradients of shape functions to real element
const auto S = geo.jacobianInverseTransposed(ip.position());
auto gradphi = makeJacobianContainer(lfsu);
for (size_t i=0; i<lfsu.size(); i++)
  S.mv(gradphihat[i][0],gradphi[i][0]);
```  

Note that, as explained in tutorial00, DUNE allows basis functions in general to be vector valued. Therefore `gradphi[i][0]` contains the gradient (with $d$ components) of the component 0 of basis function number $i$.

Now $\nabla u_h$ can be computed

```c++
Dune::FieldVector<RF,dim> gradu(0.0);
for (size_t i=0; i<lfsu.size(); i++)
  gradu.axpy(x(lfsu,i),gradphi[i][0]);
```

and we are in the position to finally compute the residual contributions:

```c++
// integrate (grad u)*grad phi_i + q(u)*phi_i
auto factor = ip.weight()*
  geo.integrationElement(ip.position());
auto q = param.q(u);
for (size_t i=0; i<lfsu.size(); i++)
  r.accumulate(lfsu,i,(gradu*gradphi[i][0]+
                       q*phihat[i])*factor);
}
```

<font color = 'red'> Ganzer Absatz sinnvoll/sinnvoll hier? </font>
Now with these definitions in place the evaluation of the algebraic residual is
\begin{equation}
R(z) = 
\sum_{T\in\mathcal{T}_h} R_T^T \mathcal{R}_T^V(R_T z)
  + \sum_{T\in\mathcal{T}_h} R_T^T \mathcal{L}_T^V
 + \sum_{F\in\mathcal{F}_h^{\partial\Omega}\cap\Gamma_N} R_T^T \mathcal{L}_F^B
\label{eq:FinalResidualEvaluation}
\end{equation}

The Jacobian of the residual is
\begin{equation*}
(J(z))_{i,j} = \frac{\partial R_i}{\partial z_j} (z) =
\sum_{(T,m,n) : (T,m)\in C(i) \wedge (T,n)\in C(j)} \frac{\partial (\mathcal{R}_T^V)_m}{\partial z_n}
(R_T z)
\end{equation*}
Note that:
1. Entries of the Jacobian can be computed element by element.
2. The derivative is independent of the $\lambda$-terms as they only depend on the test functions.
3. In the implementation below the Jacobian is computed numerically by finite differences. This can be achieved automatically by deriving from an additional base class.


# Exercises

## Warming Up

1. Try out different polynomial degrees and values for $\eta$. The parameter $\eta$ can be changed in the ini file 'tutorial01.ini'. The polynomial degree can be changed from within the notebook.
Here are some suggestions that could be interesting:
 - use higher values for $\eta$
 - Try all combinations of `degree=1|2` and `subsampling=1|2`.  Look at your solutions using paraview and the `warp by scalar` filter.  You can see the underlying grid by choosing `surface with edges` instead of `surface` in the paraview drop down menu. How does subsampling change the output?
 <font color='red'>(run the code with the following settings-check compatibility) </font>

- *example for influence of nonlinearity $\eta$ on solution. 
  Chosen parameters: `degree = 1` `uggrid`*
<table>
  <tr>
    <td><img src="images/eta0ug.png" ></td>
    <td><img src="images/eta10ug.png" ></td> 
    <td><img src="images/eta100ug.png"></td>
  </tr>
</table>

- *example for different choices of degree and subsampling for $\eta = 10$*
<table>
  <tr>
    <td><img src="images/eta100WbS11nogrid.png" ></td>
    <td><img src="images/eta100WbS21nogrid.png" ></td> 
    <td><img src="images/eta100WbS22nogrid.png"></td>
  </tr>
  <tr>
    <td><img src="images/eta100WbS11withgrid.png" ></td>
    <td><img src="images/eta100WbS21withgrid.png" ></td> 
    <td><img src="images/eta100WbS22withgrid.png"></td>
  </tr>
</table>

<font color = 'green'>
*The subsampling divides each cell into four additional cells, however just virtually for the visualiztation, the actual grid stays the same. As paraview visualizes either the value of a cell or the value of the four vertices, a further subdivision for the visualization increases the accuracy when polynomials of higher order than one are employed. This is especially important, when paraview is used for the post processing of data.*   
</font>
<font color= 'red'>
To much subsampling doesn't give any extra information, where come the extra costs into play?
</font>


2. Try to use `yasp` grid instead of `ug` grid and visualize the solution with paraview. <font color = 'red'> You can use either the structured grid factory as presented in the grid-exercise or construct it directly by using the following:</font>

```c++
std::shared_ptr<Grid> gridp = std::make_shared<Grid>(L,N);
```

- *The solution can be found within the notebook [(go to solution)](#yasp)*.

3. Try use a 3D grid implementation. For ug grid, another gmsh file needs to be provided through the ini file.

 - *change the dimension to 3 at the beginning of the notebook [(go to cell)](#dim)*.
 - *for ug: use unitcube.msh instead of unitsquare.msh, i.e. change section of ini file that is read from `grid.twod` to `grid.threed`.*

```c++
using Grid = Dune::UGGrid<dim>;
std::string filename = ptree.get("grid.threed.filename",  //3D
                                 "unitcube.msh");
Dune::GridFactory<Grid> factory;
Dune::GmshReader<Grid>::read(factory,filename,true,true);
std::unique_ptr<Grid> gridp(factory.createGrid());
Dune::Timer timer;
gridp->globalRefine(refinement);
std::cout << "Time for mesh refinement " << timer.elapsed()
          << " seconds" << std::endl;
using GV = Grid::LeafGridView;
using DF = Grid::ctype;
GV gv = gridp->leafGridView();
```

 - *for yasp: add third entry to L and N*
 
```c++
using Grid = Dune::YaspGrid<dim>;
using DF = Grid::ctype;
Dune::FieldVector<DF,dim> L;

L[0] = ptree.get("grid.structured.LX",(double)1.0);
L[1] = ptree.get("grid.structured.LY",(double)1.0);
L[2] = ptree.get("grid.structured.LZ",(double)1.0);  //3D
std::array<int,dim> N;
N[0] = ptree.get("grid.structured.NX",(int)10);
N[1] = ptree.get("grid.structured.NY",(int)10);
N[2] = ptree.get("grid.structured.NZ",(int)10);  //3D

std::shared_ptr<Grid> gridp = std::make_shared<Grid>(L,N);
gridp->globalRefine(refinement);
using GV = Grid::LeafGridView;
GV gv=gridp->leafGridView();
```

- *example 3D output*
<img src="images/threed.png" width=40%>

4. It is easy to implement different nonlinearities. Use $q(u)=\exp(\eta u)$ by adjusting the file `problem.hh`.

 - *changes in 'problem.hh'*:
 ```c++
 Number q (Number u) const
  {
    // return eta*u*u;
    return std::exp(eta*u);
  }

  //! derivative of nonlinearity
  Number qprime (Number u) const
  {
    // return 2*eta*u;
    return eta*std::exp(eta*u);
  }
```

<img src="images/newnonlin.png" width = 50%>

5. Go back to $q(u)=\eta u^2$.  Now we want to see how good our approximation is.  Change the function $f(x)$ in the file  problem.hh to $f(x)=-2d+\eta(\sum_{i=1}^d(x)_i^2)^2$ where $d$ is the dimension (and therefore size of $x$). Then $u(x)=\sum_{i=1}^d(x)_i^2=g(x)$ is the exact solution.  Visualize the exact solution like it is done in tutorial 00. Start with `degree = 1`, `eta = 10` and `subsampling = 1`. Use paraview to see how the maximal error $\max|u-u_h|$ behaves for different refienement levels `refinement=1|...|5`. Additionally you can calculate the L2-norm of the error, as before in tutorial 00. Then try again for `degree=2`.  What happens here?

 - *change of function f in `problem.hh`*:
 ```c++
  //! right hand side
  template<typename E, typename X>
  Number f (const E& e, const X& x) const
  {
    // return -2.0*x.size();
    
    // right hand side where g(x) is exact solution
    auto global = e.geometry().global(x);
    Number s=0.0;
    for (std::size_t i=0; i<global.size(); i++) s+=global[i]*global[i];
      
    return -2.0*x.size()+eta*s*s;     
  }
```


- *The visualization for the exact solution is embedded in the notebook [(go to solution)](#exact).*

- *The following table shows the maximum norm,which is given as $||f||_{\max} := \max|f(x)|$ and the L2-norm as $||f||_{0,\Omega} = \left( \int_{\Omega} f(x)^2 dx \right)^{1/2} $ for different mesh refinements. The L2 error is depicted in the graph on the right side.*

<table style = "width:100%"> 
 <tr>
  <td>
    <table  style="border:1px dotted; width:100%">
       <tr style="border:1px dotted">
        <th> refinement </th>
        <th> ${||u-u_h||}_\max$ </th> 
        <th> ${||u-u_h||}_{0,\Omega}$ </th>
      </tr>
      <tr>
        <td> 1 </td>
        <td> 0.000784874 </td> 
        <td> 0.000232161 </td>
      </tr>
       <tr>
        <td> 2 </td>
        <td> 0.000241339 </td> 
        <td> 5.51530598e-05 </td>
      </tr>
      <tr>
        <td> 3 </td>
        <td> 7.14064e-05 </td> 
        <td> 1.35100333e-05</td> 
      </tr>
       <tr>
        <td> 4 </td>
        <td> 2.18749e-05 </td> 
        <td> 3.35459387e-06 </td> 
      </tr>
      <tr>
        <td> 5 </td>
        <td> 6.61612e-06 </td> 
        <td> 8.37383425e-07</td> 
      </tr>
    </table>
   </td>  
   <td>
      <img src="images/L2norm01.png" style = "width:1000%" align = "top">
   </td>
  </tr>
</table>

- <font color = 'red'> *For `degree = 2` the finite element solution $u_h$ matches the exact solution $u$*</red>

## Nitsche's Method for weak Dirichlet Boundary Conditions

In this exercise we want to implement Dirichlet boundary conditions in a weak sense by using Nitsche's method.  Instead of incorporating the Dirichlet boundary condition into the Ansatzspace we modify the residual:
  \begin{align*}
    r^{\text{Nitsche}}(u,v) &= \int_\Omega \nabla u \cdot \nabla v + (q(u)-f)v\,dx + \int_{\Gamma_N} jv\,ds \\
    &\quad - \int_{\Gamma_D} \nabla u \cdot\nu v\,ds - \int_{\Gamma_D} (u-g)\nabla v \cdot\nu\,ds
    + \eta_{stab} \int_{\Gamma_D} (u-g)v\,ds.
  \end{align*}
Here $\eta_{stab}$ denotes a stabilization parameter that should be equal to $\eta_{stab}=c/h$ for a constant $c>0$ large enough. This stabilization term is needed to ensure coercivity of the bilinear form.

In order to implement this method you have to do the following:

 - include the new file `nitschenonlinearpoissonfem` instead of `nonlinearpoissonfem`
 - you have to turn of the constraints. The code is already there you just have to comment/uncomment the parts marked with
```c++
//== Exercise 2 {
...
// ...
//== }
```

- By changing these lines you use no constraints, an empty constraints container and construct the grid operator without constraints.  Besides that you use the new `NitscheNonlinearPoissonFEM` local operator that expects the stabilization parameter $\eta_{stab}$.

- the key part is adding the `alpha_boundary` method to the new local operator in the file `nitschenonlinearpoissonfem.hh`. Take a close look at the `lambda_volume`, `lambda_boundary` and `alpha_volume` methods and you should be on your way.

 *Hint:* The code for generating the transformation is already there:
 
```c++
// transform gradients of shape functions to real element
const auto S = geo_inside.jacobianInverseTransposed(local);
```

<font color = 'red'> is this hint actually usefull? </font>

When you have done all that test your implementation.  Use the test case from exercise 1 with $f(x)=-2d+\eta(\sum_{i=1}^d(x)_i^2)^2$ and exact solution $u(x)=\sum_{i=1}^d(x)_i^2=g(x)$ and compare it to your approximation.

Introduce the parameter <font color='red'> how to test the above without this parameter?</font>
```ini
[fem]
stab = 100
```
in the ini file and look at the maximal error $\max|u-u_h|$ for `stab=10|100|1000.`

 - *solution for method `alpha_boundary`:*
 ```c++
   //! boundary integral depending on test and ansatz functions
  template<typename IG, typename LFSU, typename X,
           typename LFSV, typename R>
  void alpha_boundary (const IG& ig, const LFSU& lfsu, const X& x,
                     const LFSV& lfsv, R& r) const
  {
    // evaluate boundary condition type
    auto localgeo = ig.geometryInInside();
    auto facecenterlocal =
      referenceElement(localgeo).position(0,0);
    bool isdirichlet=param.b(ig.intersection(),facecenterlocal);

    // skip rest if we are _not_ on Dirichlet boundary
    if (!isdirichlet) return;

    // types & dimension
    const int dim = IG::Entity::dimension;
    using RF = decltype(Dune::PDELab::makeZeroBasisFieldValue(lfsu));

    // select quadrature rule
    auto geo = ig.geometry();
    const int order = incrementorder+
      2*lfsu.finiteElement().localBasis().order();
    auto rule = Dune::PDELab::quadratureRule(geo,order);

    // geometry of inside cell
    auto geo_inside = ig.inside().geometry();

    // loop over quadrature points
    for (const auto& ip : rule)
      {
        // quadrature point in local coordinates of element
        auto local = localgeo.global(ip.position());

        // evaluate basis functions
        auto& phihat = cache.evaluateFunction(local,
                             lfsu.finiteElement().localBasis());

        // evaluate u
        RF u=0.0;
        for (size_t i=0; i<lfsu.size(); i++)
          u += x(lfsu,i)*phihat[i];

        // evaluate gradient of shape functions
        auto& gradphihat = cache.evaluateJacobian(local,
                             lfsu.finiteElement().localBasis());

        // transform gradients of shape functions to real element
        const auto S = geo_inside.jacobianInverseTransposed(local);
        auto gradphi = makeJacobianContainer(lfsu);
        for (size_t i=0; i<lfsu.size(); i++)
          S.mv(gradphihat[i][0],gradphi[i][0]);

        // compute gradient of u
        Dune::FieldVector<RF,dim> gradu(0.0);
        for (size_t i=0; i<lfsu.size(); i++)
          gradu.axpy(x(lfsu,i),gradphi[i][0]);

        // get unit outer normal vector and g
        auto n = ig.unitOuterNormal(ip.position());
        auto g = param.g(ig.intersection(),ip.position());

        // integrate -(grad u)*n * phi_i + q(u)*phi_i
        auto factor = ip.weight()*
          geo.integrationElement(ip.position());
        for (size_t i=0; i<lfsu.size(); i++)
          r.accumulate(lfsu,i,(-(gradu*n)*phihat[i]-
                               (u-g)*(gradphi[i][0]*n)+
                               stab*(u-g)*phihat[i])*factor);
      }
  }
```

 - $\eta_{stab}$ needs to be large enough to ensure coercivity <font color='red'> important </font>
<table>
  <tr>
    <td><img src="images/stab10.png" ></td>
    <td><img src="images/stab100.png" ></td> 
    <td><img src="images/stab1000.png"></td>
  </tr>
</table>

- <font color='red'> Use cases/advantages of either method? </font>
<table>
  <tr>
    <td><img src="images/DBCAnsatz.png" width= 90%></td>
    <td><img src="images/DBCresidual.png" ></td> 
  </tr>
</table>